In [10]:
from balm.config import BalmConfig, BalmMoEConfig#, BalmExpertChoiceMoEConfig
from balm.data import load_dataset, DataCollator
from balm.models import (
    BalmForMaskedLM,
    BalmModel,
    BalmMoEForMaskedLM,
    #BalmExpertChoiceMoEForMaskedLM,
    #BalmHybridMoEForMaskedLM,
    #BalmMoERoPEForMaskedLM,
)
from balm.config import BalmConfig
from balm.tokenizer import Tokenizer
from balm.train import Trainer

In [2]:
tokenizer = Tokenizer(vocab="./vocab.json")

In [3]:
def remove_sep(txt):
    return txt.replace("</s>", "<cls><cls>")


data_files = {
    "train": "./balm/test_data/test_1k.txt",
    "test": "./balm/test_data/test_1k.txt",
    "eval": "./balm/test_data/test_1k.txt",
}

# data_files = {
#     "train": "../train-test-eval_paired/train.txt",
#     "test": "../train-test-eval_paired/test.txt",
#     "eval": "../train-test-eval_paired/eval.txt",
# }

# data_files = {
#     "train": "../jaffe-plusHD_clust0.9_split/train.txt",
#     "test": "../jaffe-plusHD_clust0.9_split/test.txt",
#     "eval": "../jaffe-plusHD_clust0.9_split/eval.txt",
# }

dataset = load_dataset("text", data_files=data_files, preprocess_fn=remove_sep)

In [4]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["text"],
        padding=True,
        truncation=True,
        max_length=320,
    ),
    remove_columns="text"
)

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
collator = DataCollator(tokenizer=tokenizer)

In [12]:
# # matched to ESM-2 8M
config = BalmConfig(
    embed_dim=320,
    ffn_dim=320 * 4,
    num_layers=6,
    num_heads=20,
    vocab_size=tokenizer.vocab_size,
)
model = BalmForMaskedLM(config=config)

In [13]:
model.num_parameters

6294113

In [16]:
trainer = Trainer(
    model=model,
    data_collator=collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
    output_dir="./training_runs/save_tests",
    epochs=1,
    logging_steps=5,
    eval_steps=100,
    warmup_steps=10,
    # save_steps=15,
    per_device_train_batch_size=32,
    # use_cpu=True,
    # use_wandb=True,
    # wandb_project="test_wandb_logging",
    # wandb_entity="bryanbriney",
    run_name="save_test_001",
    deepspeed=True,
    deepspeed_config='deepspeed_config_stage1.json'
)


In [17]:
trainer.train()

[2024-05-28 15:44:46,084] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible
[2024-05-28 15:44:47,188] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.2, git-hash=unknown, git-branch=unknown
[2024-05-28 15:44:47,189] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-28 15:44:47,189] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist

ModuleNotFoundError: No module named 'mpi4py'